In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D, Activation, Dropout, Flatten, Dense
from keras.applications import ResNet50
from keras import optimizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import pandas as pd
from tqdm import tqdm
import gc
import cv2

/home/faisal/anaconda3/envs/ai/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
import numpy as np
import tensorflow as tf
import random as rn

# The below is necessary in Python 3.2.3 onwards to
# have reproducible behavior for certain hash-based operations.
# See these references for further details:
# https://docs.python.org/3.4/using/cmdline.html#envvar-PYTHONHASHSEED
# https://github.com/keras-team/keras/issues/2280#issuecomment-306959926

import os
os.environ['PYTHONHASHSEED'] = '0'
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.

np.random.seed(42)

# The below is necessary for starting core Python generated random numbers
# in a well-defined state.

rn.seed(12345)

# Force TensorFlow to use single thread.
# Multiple threads are a potential source of
# non-reproducible results.
# For further details, see: https://stackoverflow.com/questions/42022950/which-seeds-have-to-be-set-where-to-realize-100-reproducibility-of-training-res

session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

from keras import backend as K

# The below tf.set_random_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see: https://www.tensorflow.org/api_docs/python/tf/set_random_seed

tf.set_random_seed(1234)

sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)


In [4]:
img_width, img_height = (224, 224)

train_data_dir = './data/train_img/'
test_data_dir = './data/test_img/'
epochs = 10
batch_size = 128

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)


In [5]:
resnet_model = ResNet50(include_top=False, input_shape=input_shape)

In [6]:
def get_model():
# add a global spatial average pooling layer
    x = resnet_model.output
    x = GlobalAveragePooling2D()(x)
    predictions = Dense(85, activation='sigmoid')(x)
    model = Model(inputs=resnet_model.input, outputs=predictions)
    
    return model

In [7]:
model = get_model()

We'll start with training the head(last layer) only as that layer is initialized randomaly and we don't want to affect the other layers weights as while backpropogation. 

In [8]:
#train only last layer
for layer in model.layers[:-1]:
    layer.trainable = False

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [9]:
def augment_brightness(img):
    img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    random_bright = .25 + np.random.uniform()
    img[:,:,2] = img[:,:,2] * random_bright
    img = cv2.cvtColor(img, cv2.COLOR_HSV2RGB)
    return img

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(preprocessing_function=augment_brightness,
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2,
                                   rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True, 
                                   rotation_range = 20,
                                   channel_shift_range=0.2,
#                                    zca_whitening=True
                                   )

val_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

In [10]:
train = pd.read_csv('./data/train.csv', index_col=0)
test = pd.read_csv('./data/test.csv')
attributes = pd.read_csv('./data/attributes.txt', delimiter='\t', header=None, index_col=0)
classes = pd.read_csv('./data/classes.txt', delimiter='\t', header=None, index_col=0)

In [13]:
def get_imgs(src, df, labels = False):
    if labels == False:
        imgs = []    
        files = df['Image_name'].values
        for file in tqdm(files):
            im = cv2.imread(os.path.join(src, file))
            im = cv2.resize(im, (img_width, img_height))
            imgs.append(im)
        return np.array(imgs)
    else:
        imgs = []
        labels = []
        files = os.listdir(src)
        for file in tqdm(files):
            im = cv2.imread(os.path.join(src, file))
            im = cv2.resize(im, (img_width, img_height))
            imgs.append(im)
            labels.append(df.loc[file].values)
        return np.array(imgs), np.array(labels)

In [14]:
train_imgs, train_labels = get_imgs(train_data_dir, train, True)

  2%|▏         | 221/12600 [00:09<09:13, 22.35it/s]/home/faisal/anaconda3/envs/ai/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
100%|██████████| 12600/12600 [08:41<00:00, 24.16it/s]


In [15]:
#train val split
X_tra, X_val, y_tra, y_val = train_test_split(train_imgs, train_labels, test_size = 3000, random_state = 222)


In [16]:
gc.collect()

108515

In [17]:
X_tra.shape

(9600, 224, 224, 3)

In [18]:
import pickle

data_save = {}
data_save['train_x'] = X_tra
data_save['train_y'] = y_tra
data_save['valid_x'] = X_val
data_save['valid_y'] = y_val

with open('data.pickle', 'wb') as f:
    pickle.dump(data_save, f, pickle.HIGHEST_PROTOCOL)

In [19]:
train_datagen.fit(X_tra)
# val_datagen.fit(X_val)

In [20]:
val_datagen.fit(X_val)

In [21]:
def fmeasure(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))

We're going to train our model with SGD and very low learning rate

In [19]:
early_stp = EarlyStopping(monitor="val_loss", mode='min', patience=5)
model_ckpt = ModelCheckpoint('resnet_1_layer.h5', save_best_only=True, mode='min', monitor='val_loss', verbose=1, save_weights_only=True)

# opt = optimizers.SGD(lr=0.001, decay = 1e-6, momentum = 0.9, nesterov = True)
opt = optimizers.Adam(lr=1e-4)
model.compile(opt, loss = 'binary_crossentropy', metrics=['accuracy', fmeasure])
model.reset_states()

In [22]:
# batch_size=
model.fit_generator(train_datagen.flow(X_tra, y_tra, batch_size=batch_size),                     
                    steps_per_epoch=len(X_tra) / batch_size, epochs=100,
                    validation_data=val_datagen.flow(X_val, y_val, batch_size=batch_size), 
                    validation_steps = len(X_val)/batch_size, callbacks=[early_stp, model_ckpt], workers = 10, max_queue_size=20)

Epoch 1/100
75/75 [==============================] - 33s 441ms/step - loss: 0.5469 - acc: 0.7224 - fmeasure: 0.6174 - val_loss: 0.6548 - val_acc: 0.6097 - val_fmeasure: 0.4820

Epoch 00001: val_loss improved from inf to 0.65480, saving model to resnet_1_layer.h5
Epoch 2/100
75/75 [==============================] - 27s 364ms/step - loss: 0.4226 - acc: 0.8110 - fmeasure: 0.7281 - val_loss: 0.6413 - val_acc: 0.6301 - val_fmeasure: 0.5022

Epoch 00002: val_loss improved from 0.65480 to 0.64134, saving model to resnet_1_layer.h5
Epoch 3/100
75/75 [==============================] - 28s 367ms/step - loss: 0.3676 - acc: 0.8442 - fmeasure: 0.7772 - val_loss: 0.6306 - val_acc: 0.6405 - val_fmeasure: 0.5053

Epoch 00003: val_loss improved from 0.64134 to 0.63057, saving model to resnet_1_layer.h5
Epoch 4/100
75/75 [==============================] - 28s 368ms/step - loss: 0.3321 - acc: 0.8641 - fmeasure: 0.8069 - val_loss: 0.6230 - val_acc: 0.6544 - val_fmeasure: 0.5133

Epoch 00004: val_loss impr

KeyboardInterrupt: 

In [25]:
model = get_model()

#train only last 20 layer
for layer in model.layers:
    layer.trainable = True

opt = optimizers.Adam(lr=8e-7)
# opt = optimizers.SGD(lr=0.001, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(opt, loss = 'binary_crossentropy', metrics=['accuracy', fmeasure])
    
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [26]:
early_stp = EarlyStopping(monitor="val_loss", mode='min', patience=5)
model_ckpt = ModelCheckpoint('resnet_all_layers.h5', save_best_only=True, mode='min', monitor='val_loss', verbose=1, save_weights_only=True)

# model.load_weights('resnet_1_layer.h5')
model.load_weights('resnet_all_layers.h5')


In [27]:
batch_size = 32
model.fit_generator(train_datagen.flow(X_tra, y_tra, batch_size=batch_size),                     
                    steps_per_epoch=len(X_tra) / batch_size, epochs=100,
                    validation_data=val_datagen.flow(X_val, y_val, batch_size=batch_size), 
                    validation_steps = len(X_val)/batch_size, callbacks=[early_stp, model_ckpt], workers = 10, max_queue_size=20)

Epoch 1/100
300/300 [==============================] - 85s 282ms/step - loss: 0.0340 - acc: 0.9896 - fmeasure: 0.9857 - val_loss: 0.0664 - val_acc: 0.9759 - val_fmeasure: 0.9671

Epoch 00001: val_loss improved from inf to 0.06635, saving model to resnet_all_layers.h5
Epoch 2/100
300/300 [==============================] - 77s 255ms/step - loss: 0.0342 - acc: 0.9896 - fmeasure: 0.9857 - val_loss: 0.0661 - val_acc: 0.9762 - val_fmeasure: 0.9674

Epoch 00002: val_loss improved from 0.06635 to 0.06614, saving model to resnet_all_layers.h5
Epoch 3/100
300/300 [==============================] - 77s 256ms/step - loss: 0.0332 - acc: 0.9900 - fmeasure: 0.9863 - val_loss: 0.0665 - val_acc: 0.9762 - val_fmeasure: 0.9673

Epoch 00003: val_loss did not improve
Epoch 4/100
300/300 [==============================] - 76s 252ms/step - loss: 0.0325 - acc: 0.9903 - fmeasure: 0.9867 - val_loss: 0.0660 - val_acc: 0.9763 - val_fmeasure: 0.9674

Epoch 00004: val_loss improved from 0.06614 to 0.06597, saving m

In [28]:
test_imgs = get_imgs(test_data_dir, test)

  8%|▊         | 422/5400 [00:09<01:57, 42.30it/s]/home/faisal/anaconda3/envs/ai/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
100%|██████████| 5400/5400 [01:17<00:00, 69.66it/s]


In [29]:
test_datagen.fit(test_imgs)

In [30]:
pred = model.predict_generator(test_datagen.flow(test_imgs, batch_size=512, shuffle=False), verbose=1, workers=8)

11/11 [==============================] - 17s 2s/step


In [31]:
sub = pd.read_csv('./data/sample_submission.csv')
sub.iloc[:, 1:] = pred.round().astype(int)
sub.head()

,Image_name,attrib_01,attrib_02,attrib_03,attrib_04,attrib_05,attrib_06,attrib_07,attrib_08,attrib_09,...,attrib_76,attrib_77,attrib_78,attrib_79,attrib_80,attrib_81,attrib_82,attrib_83,attrib_84,attrib_85
0,Image-1.jpg,0,0,0,0,0,1,0,0,1,...,0,0,0,1,1,0,0,1,0,1
1,Image-2.jpg,0,1,0,1,0,1,1,0,0,...,0,1,0,1,0,0,0,0,1,1
2,Image-3.jpg,0,0,0,1,0,0,1,0,0,...,1,0,0,1,0,0,0,1,0,1
3,Image-4.jpg,1,0,0,0,1,1,1,0,0,...,0,1,1,1,0,0,0,0,0,1
4,Image-5.jpg,0,0,0,1,0,1,0,0,0,...,0,0,0,1,1,0,0,1,0,0


In [32]:
sub.to_csv('submission.csv', index=False)

In [33]:
sub.shape

(5400, 86)

## Final Thoughts

This submission should get you around $\approx$0.80 on the LB. and if you've noticed that our last epochs val_fmeasure is the same so it means that our val set is represantion of the test set and you can train for many epochs with EarlyStopping without worring about overfitting on val or train set.

### How to improve from here?
You can change many things which will let you get higher LB score. Following is a small list
<ul>
    <li> Change the Image size to bigger number </li>
    <li> Increse the number of epoch in the fully trainable network($2^{nd}$ training) </li>
    <li> Use diffrent architechure. you'll get more info on that [here](keras.io/applications/)</li>
    <li> If nothing works ensemble is your best friend </li>
</ul>

I'll try to keep improving this notebook. Feel free to contribuite.

Thanks
